In [1]:
!pip install pytesseract pdf2image
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-hin
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,606 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [44]:
import pytesseract
from pdf2image import convert_from_path
import csv
import re

pdf_path = '/content/EROLLGEN-S20-62-FinalRoll-Revision1-HIN-1-WI.pdf'

# Convert PDF to images
images = convert_from_path(pdf_path)

# Set the language for Tesseract OCR
tesseract_language = 'hin'  # Hindi language code

# Initialize a list to store extracted information
extracted_info = []

# Define the keywords to extract
keywords = ["निर्वाचक का नाम", "पिता का नाम", "पति का नाम", "मकान संख्या", "उम्र", "लिंग"]

# Extract text from images using Tesseract OCR
for img in images:
    extracted_text = pytesseract.image_to_string(img, lang=tesseract_language)
    lines = extracted_text.strip().splitlines()
    extracted_info.extend(lines)

# Organize extracted information into rows
rows = []
current_row = {name: None for name in keywords}

for line in extracted_info:
    for keyword in keywords:
        if keyword in line:
            value = line.replace(keyword, "").strip(':')
            current_row[keyword] = value
            if keyword == "लिंग":  # If the last keyword is encountered, add the current row to rows list
                rows.append(current_row)
                current_row = {name: None for name in keywords}


# Clean up and format the data using regex
cleaned_and_filtered_rows = []
for row in rows:
    row["उम्र"] = re.search(r"\d+", row["उम्र"]).group() if row["उम्र"] else None
    if row["उम्र"] and len(row["उम्र"]) != 2:
        row["उम्र"] = row["उम्र"] + "1"
    row["लिंग"] = "पुरुष" if "पुरुष" in row["लिंग"] else "महिला"
    if row["मकान संख्या"] is not None:
        match = re.search(r"\d+", row["मकान संख्या"])
        row["मकान संख्या"] = match.group() if match else None
    if row["निर्वाचक का नाम"] and len(row["निर्वाचक का नाम"]) <= 20 and 'निर्वाचक' not in row["निर्वाचक का नाम"]:
        cleaned_and_filtered_rows.append(row)

# Define CSV file path
csv_file_path = 'extracted_info.csv'

# Write the cleaned and filtered data to a CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.DictWriter(csvfile, fieldnames=keywords)
    csv_writer.writeheader()
    csv_writer.writerows(cleaned_and_filtered_rows)

print("Cleaned and filtered data has been extracted and saved to", csv_file_path)


Cleaned and filtered data has been extracted and saved to extracted_info.csv


In [48]:
import pandas as pd
df= pd.read_csv("/content/extracted_info.csv")
df.head(10)

,निर्वाचक का नाम,पिता का नाम,पति का नाम,मकान संख्या,उम्र,लिंग
0,: लालाराम,मामचन्द,NaN,2.0,59,पुरुष
1,: सुरेश कुमार यादव,छाजूराम,NaN,22.0,43,पुरुष
2,: शर्मिला,NaN,अभय सिह,4.0,32,महिला
3,: ओमली,NaN,लालाराम,2.0,54,महिला
4,: श्योताज,गणपतराम,NaN,4.0,79,पुरुष
5,: सावित्री,NaN,श्योताज,4.0,69,महिला
6,: देवेंद्र कुमार,लालाराम,NaN,2.0,22,पुरुष
7,: अभयसिंह,श्योताज,NaN,4.0,25,पुरुष
8,: अशोक,श्योत्ाज सिंह,NaN,4.0,33,पुरुष
9,: प्रदीप कुमार,राजेन्द्र प्रसाद,NaN,8.0,27,पुरुष
